# Estructura del mundo de Wumpus

----

### El Mundo de Wumpus

El mundo de Wumpus es una cueva compuesta por habitaciones conectadas por pasadizos. Acechando en algún lugar de la cueva se encuentra el terrible Wumpus, una bestia que devora a cualquiera que entre en su habitación. El Wumpus puede ser abatido por un agente, pero el agente solo tiene una flecha. Algunas habitaciones contienen pozos sin fondo que atraparán a cualquiera que entre en ellas (excepto al Wumpus, que es demasiado grande para caer en ellos). La única característica redentora de este sombrío entorno es la posibilidad de encontrar un montón de oro.

#### Medida de desempeño
+1000 por salir de la cueva con el oro, -1000 por caer en un pozo o ser devorado por el Wumpus, -1 por cada acción tomada, y -10 por usar la flecha. El juego termina cuando el agente muere o cuando el agente sale de la cueva.

#### Entorno
Una cuadrícula de habitaciones, con paredes que rodean la cuadrícula. El agente siempre comienza en el cuadrado etiquetado como [1,1], mirando hacia el este. Las ubicaciones del oro y del Wumpus se eligen aleatoriamente, con una distribución uniforme, de entre los cuadrados distintos al cuadrado de inicio. Además, cada cuadrado, aparte del inicio, puede contener un pozo, con una probabilidad de 0.2.

#### Actuadores
El agente puede avanzar (Forward), girar a la izquierda (TurnLeft) en 90 grados, o girar a la derecha (TurnRight) en 90 grados. El agente muere una muerte miserable si entra en un cuadrado que contiene un pozo o un Wumpus vivo (es seguro, aunque maloliente, entrar en un cuadrado con un Wumpus muerto). Si un agente intenta avanzar y choca con una pared, entonces el agente no se mueve. La acción "Agarrar" (Grab) se puede usar para recoger el oro si está en el mismo cuadrado que el agente. La acción "Disparar" (Shoot) se puede usar para disparar una flecha en línea recta en la dirección en la que mira el agente. La flecha continúa hasta que golpea (y por lo tanto mata) al Wumpus o golpea una pared. El agente solo tiene una flecha, por lo que solo la primera acción de disparo tiene algún efecto. Finalmente, la acción "Escalar" (Climb) se puede usar para salir de la cueva, pero solo desde el cuadrado [1,1].

#### Sensores
El agente tiene cinco sensores, cada uno de los cuales proporciona un solo bit de información:

En los cuadrados directamente adyacentes (no en diagonal) al Wumpus, el agente percibirá un "Hedor" (Stench).
En los cuadrados directamente adyacentes a un pozo, el agente percibirá una "Brisa" (Breeze).
En el cuadrado donde está el oro, el agente percibirá un "Brillo" (Glitter).
Cuando un agente choca con una pared, percibirá un "Golpe" (Bump).
Cuando el Wumpus es abatido, emite un "Grito" (Scream) que se puede percibir en cualquier lugar de la cueva.

### Tipos y Globales

Se definen las opciones de strings como `enums`, las 4 orientaciones, las 5 acciones y las 6 percepciones (que incluyen el no percibir nada)

In [61]:
from typing import Literal, Tuple

# Type definitions for orientations, actions, and percepts
Orientation = Literal['North', 'East', 'South', 'West']
Action = Literal['Forward', 'TurnLeft', 'TurnRight', 'Grab', 'Shoot']
Percept = Literal['Stench', 'Breeze', 'Glitter', 'Bump', 'Scream', 'None']

# Orientation order to manage turning left and right
ORIENTATION_ORDER: list[Orientation] = ['North', 'East', 'South', 'West']

### Conocimiento Base

Almacena las sentencias del agente, permite hacer queries de acciones o perceptos realizados

In [62]:
class BaseKnowledge:
    sentences: list[str]

    def __init__(self):
        self.sentences = []

    def tell(self, sentence: str):
        """Stores a sentence in the agent's knowledge base."""
        self.sentences.append(sentence)

    def ask(self, query: str) -> str | None:
        """Returns the most recent sentence that matches the query."""
        for sentence in reversed(self.sentences):
            if query in sentence: return sentence

    def __str__(self) -> str:
        """String representation of the agent's knowledge base."""
        return 'Sentences:\n' + '\n'.join(self.sentences)

### Funciones Ayudantes

`map_to_matrix` convierte las coordenadas del mundo de Wumpus en coordenadas de matriz de Python y `get_position_variation` determina el cambio en las coordenadas dependiendo de cual sea la orientacion del agente

In [63]:
def map_to_matrix(position: Tuple[int, int], n: int) -> Tuple[int, int]:
    """Converts map coordinates to matrix coordinates for accessing MAP."""
    col, row = position
    mat_col = col
    mat_row = n - row - 1
    return (mat_row, mat_col)

def get_position_variation(orientation: Orientation) -> Tuple[int, int]:
    dx, dy = (0, 0)  # Change in position based on orientation
    match orientation:
        case 'North': dy += 1
        case 'East': dx += 1
        case 'South': dy -= 1
        case 'West': dx -= 1
    return (dx, dy)

### Agente de Wumpus

El agente inicializa sus atributos y define un conocimiento inicial afirmando que no hay ni Pit ni Wumpus en la pocision inicial

Define las 5 acciones que toma el agente, maneja el paso del tiempo y la estructura de las sentencias

In [68]:
class WumpusAgent:
    base: BaseKnowledge
    position: Tuple[int, int]  # (Column, Row)
    orientation: Orientation
    has_gold: bool
    has_arrow: bool
    time: int

    def __init__(self):
        self.base = BaseKnowledge() 
        self.position = (1, 1)  
        self.orientation = 'East' 
        self.has_gold = False   
        self.has_arrow = True  
        self.time = 0  

        # Tells base knowledge that there's no Wumpus or Pit at the start position
        self.base.tell(self.make_action_query('-W', (1, 1)))
        self.base.tell(self.make_action_query('-P', (1, 1)))

    def action_forward(self):
        """Moves the agent forward, checking for walls (Bump) and updating position."""
        col, row = self.position
        dx, dy = get_position_variation(self.orientation)
        y, x = map_to_matrix((col, row), len(MAP))  # Convert position to matrix coordinates
        if MAP[y - dy][x + dx] == '#':  # Check if the next position is a wall
            sentence = self.make_percept_sentence('Bump', self.time + 1)
            self.base.tell(sentence)  # Record a bump event in knowledge base
            return
        self.position = (col + dx, row + dy)  # Update position if no wall

    def action_grab(self):
        """Allows the agent to grab the gold if Glitter is perceived."""
        suppose = self.make_percept_sentence('Glitter', self.time)  # Check if Glitter is perceived
        percepts_gold = self.base.ask(suppose)
        if not percepts_gold: return  # If no gold, return

        self.has_gold = True  # Agent grabs the gold
        i, j = map_to_matrix(self.position, len(MAP))
        MAP[i][j] = MAP[i][j].replace('G', '')  # Remove Glitter from the map

    def action_shoot(self):
        """Shoots an arrow in the direction of the agent, checking for Wumpus and causing a Scream."""
        suppose = self.make_percept_sentence('Stench', self.time)  # Check for Stench
        smells = self.base.ask(suppose)
        if not smells: return  # If no Stench, do not shoot

        def kill_wumpus(x: int, y: int):
            """Helper function to kill Wumpus and record a Scream event."""
            if MAP[y][x] != 'W': return False  # No Wumpus in this location
            MAP[y][x].replace('W', '')  # Remove Wumpus from the map
            sentence = self.make_percept_sentence('Scream', self.time + 1)
            self.base.tell(sentence)  # Record a scream event
            return True

        self.has_arrow = False  # Use up the arrow
        y, x = map_to_matrix(self.position, len(MAP))
        match self.orientation:
            # Check each direction for a Wumpus, killing it if found
            case 'North':
                for i in range(y - 1, 0, -1):
                    if kill_wumpus(x, i): break
            case 'East':
                for i in range(x + 1, len(MAP)):
                    if kill_wumpus(i, y): break
            case 'South':
                for i in range(y + 1, len(MAP)):
                    if kill_wumpus(x, i): break
            case 'West':
                for i in range(x - 1, 0, -1):
                    if kill_wumpus(i, y): break

    def action_left(self):
        """Turns the agent to the left (counterclockwise)."""
        index = ORIENTATION_ORDER.index(self.orientation)
        self.orientation = ORIENTATION_ORDER[(index - 1) % len(ORIENTATION_ORDER)]  # Update orientation

    def action_right(self):
        """Turns the agent to the right (clockwise)."""
        index = ORIENTATION_ORDER.index(self.orientation)
        self.orientation = ORIENTATION_ORDER[(index + 1) % len(ORIENTATION_ORDER)]  # Update orientation

    def next_time(self):
        """Advances the time counter."""
        self.time += 1

    def make_percept_sentence(self, percept: Percept, t: int):
        """Creates a sentence representing a percept at time t."""
        return f'[t={t}] Percept: {percept}'

    def make_action_query(self, q: str, p: Tuple[int, int]):
        """Creates a query for the agent's action at time t."""
        return f'Query: {q}{p[0]}{p[1]}'

    def make_action_sentence(self, action: Action, t: int):
        """Creates a sentence representing an action at time t."""
        return f'[t={t}] Action: {action}'

    def decide_action(self, percept: list[Percept]) -> Action:
        """Decides the agent's action based on the current percepts."""
        c, r = self.position
        dx, dy = get_position_variation(self.orientation)
        rr = r + dy
        cc = c + dx
        if 'Glitter' in percept: return 'Grab'
        if 'Stench' in percept:
            query_nowump = self.base.ask(self.make_action_query('-W', (cc, rr)))
            if not query_nowump:
                return 'TurnRight'
            if self.has_arrow: return 'Shoot'
        else:
            self.base.tell(self.make_action_query('-W', (cc, rr)))
        if 'Breeze' in percept:
            query_nopit = self.base.ask(self.make_action_query('-P', (cc, rr)))
            if not query_nopit:
                self.base.tell(self.make_action_query('p', (cc, rr)))
                return 'TurnLeft'
        else:
            self.base.tell(self.make_action_query('-P', (cc, rr)))
        if 'Bump' in percept: return 'TurnRight'
        if 'Scream' in percept: return 'Forward'
        return 'Forward'  # Default action

    def execute(self, action: Action):
        """Executes the given action."""
        match action:
            case 'Forward': self.action_forward()
            case 'Grab': self.action_grab()
            case 'Shoot': self.action_shoot()
            case 'TurnLeft': self.action_left()
            case 'TurnRight': self.action_right()
            case _: self.action_forward()  # Default to moving forward

    def act(self, percept: list[Percept]) -> Action:
        """Processes percepts, determines the next action, and executes it."""
        for sensor in percept:
            if sensor == 'None': continue
            sentence = self.make_percept_sentence(sensor, self.time)
            self.base.tell(sentence)  # Store each percept in knowledge base

        # Check for Bump and Scream percepts from knowledge base
        bump = self.base.ask(self.make_percept_sentence('Bump', self.time))
        scream = self.base.ask(self.make_percept_sentence('Scream', self.time))
        if bump: percept.append('Bump')
        if scream: percept.append('Scream')

        # Ask the knowledge base if there's an inferred action for the current time
        action = self.decide_action(percept)

        # Store the action in the knowledge base and execute it
        sentence = self.make_action_sentence(action, self.time)
        self.base.tell(sentence)
        self.execute(action)

        # Move to the next time step
        self.next_time()
        return action
    
    def __str__(self) -> str:
        """String representation of the agent's current state."""
        return f'{self.position} {self.orientation}\t:: G:{int(self.has_gold)} - A:{int(self.has_arrow)}.'


### Main

Define el mapa del mundo de Wumpus como matriz y ejecuta la accion del agente hasta que este recoja el Gold

In [ ]:
# Map representation, with '#' for walls, 'S' for Stench, 'B' for Breeze, 'G' for Glitter, 'P' for Pit, 'W' for Wumpus
MAP: list[list[str]] = [
    ['#', '#', '#', '#', '#', '#'],
    ['#', 'S', ' ', 'B', 'P', '#'],
    ['#', 'W', 'SBG', 'P', 'B', '#'],
    ['#', 'S', ' ', '#', ' ', '#'],
    ['#', 'X', 'B', 'P', 'B', '#'],
    ['#', '#', '#', '#', '#', '#'],
]

def main():
    agent = WumpusAgent()
    while True:
        r, c = map_to_matrix(agent.position, len(MAP))
        sensor = MAP[r][c]  # Get the current percepts based on the agent's position
        percept = [
            'Stench' if 'S' in sensor else 'None',
            'Breeze' if 'B' in sensor else 'None',
            'Glitter' if 'G' in sensor else 'None',
            'None',  # Bump
            'None',  # Scream
        ]
        action = agent.act(percept)  # Agent decides and acts based on percepts
        print(agent, action)  # Print the agent's state and action
        if agent.has_gold: break
    print(agent.base)  # Print the agent's knowledge base

if __name__ == "__main__":
    main()  